So here's the idea right: when a company makes a big statement, whether it be an acquisition, a merger, a new product, etc whatever it usually affects the stock price in some direction. This change is usually pretty drastic, and for some outlier and small companies it can be gigantic gains. 

The idea is to exploit the **lag time** between a) when a company makes an announcement and b) when and how the market reacts. The theory is that the company makes the announcement but only few people see it, and then once that news is picked up by larger outlets more people see it and give a larger market response.

The goal is to parse various company websites for news, and see how their announcements affect the market. The end steps here are:

1. Choose industry to focus on: **Healthcare and Pharmaceuticals**
2. Identify and target smaller stocks that have the potential for rapid movement: **20/share** cap for now
3. Parse the company websites regularly to check for press releases.
4. When a new press release is made, determine if the stock will raise or fall from the news, and a confidence level.
5. ???
6. Profit!

This is obviously the full writeout of the end plan, but there's some prelim work to do first:

- Determine what the lag time is usually. If it's very low a server might have to be established.
- Develop a way to test the confidence level, will probably require more statistical knowledge than I currently have.
- Retroactive analysis at first

In [1]:
import pandas as pd
import numpy as np

# retreive health.csv, which contains all symbols, names, and recent statistics of all NASDAQ healthcare stocks
# retreived from https://www.nasdaq.com/market-activity/stocks/screener

df = pd.read_csv('health.csv')
tickers=list(df['Symbol'])

In [116]:
import requests as req
# Want to automatically retreive recent news data from tickers
# Start with one, our Ocugen Case (index 719)
ticker=tickers[719]

# Can crawl press releases on NASDAQ, sticking to just press releases to filter out irrelevant and redundant news articles
# https://www.nasdaq.com/market-activity/stocks/(TICKER)/press-releases
# First get the request of the site
url='https://www.nasdaq.com/market-activity/stocks/{}/press-releases'.format(ticker.lower())
# want to get data from a with class="quote-press-release__link"
    # underneat that is a span that has the headline

# UPDATE: Pitch this to Scrapy
# UPDATE 2: I don't know how to use scrapy but I have to learn

In [85]:
# prep for selenium loading
# want to get ticker, title and href of most recent article
test=[]

In [89]:
# Selenium is a bit heavy handed 
from selenium import webdriver

browser=webdriver.Edge("C:/Users/cukel/Downloads/edgedriver_win32/msedgedriver.exe")
browser.get(url)

a_eles = browser.find_elements_by_xpath("//a[@class='quote-press-release__link']")
s_eles = browser.find_elements_by_xpath("//a[@class='quote-press-release__link']/span")
d_eles = browser.find_elements_by_xpath("//div[@class='quote-press-release__card-date']/div[@class='quote-press-release__card-timestamp']")

n=0
title = s_eles[n].text
href = a_eles[n].get_attribute("href")
date = d_eles[n].text #offset weird, first is 0 though so that's what matters

test.append([ticker,date,title,href])

# STARTED BELOW AT 2:11 AM JUST AS A HEADS UP

In [2]:
# Time to functionize this and run this shit into the ground
# assume generated tickers from above, becomes 2 step
from selenium import webdriver #import
import numpy as np
# start the driver
browser=webdriver.Edge("C:/Users/Luke/Downloads/edgedriver_win32/msedgedriver.exe")
data=[]
# loop through the tickers
for ii in range(0,len(tickers)): # exception occured at 44
    if ii%5==0:
        print('done: '+str(ii))
    # generate and retreive url
    ticker=tickers[ii]
    url='https://www.nasdaq.com/market-activity/stocks/{}/press-releases'.format(ticker.lower())
    browser.get(url)
    # find the required elements
    a_eles = browser.find_elements_by_xpath("//a[@class='quote-press-release__link']")
    s_eles = browser.find_elements_by_xpath("//a[@class='quote-press-release__link']/span")
    d_eles = browser.find_elements_by_xpath("//div[@class='quote-press-release__card-date']/div[@class='quote-press-release__card-timestamp']")
    # apply methods to get desired info from elements
    try:
        n=0
        #title = s_eles[n].text
        href = a_eles[n].get_attribute("href")
        #date = d_eles[n].text #offset weird, first is 0 though so that's what matters
        # append to external list
        browser.get(href)
        fulltime=browser.find_elements_by_xpath("//time[@class='timestamp__date']")[n].text
        data.append([ticker,date,title,href,fulltime])
    except IndexError:
        # no press releases on the site
        data.append([ticker,np.nan,np.nan,np.nan])       
    break

done: 0


## Pull all press releases on page
It's kinda wonly, need mouse cursor in specific spot.

In [44]:
# THIS ONE pulls all press releases under a specific company
# it is based on the above code but obviously it has been modified. 

#Time to functionize this and run this shit into the ground
# assume generated tickers from above, becomes 2 step
from selenium import webdriver #import
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from msedge.selenium_tools import EdgeOptions
from msedge.selenium_tools import Edge
import numpy as np
import os
# launch edge
#driver = Edge("C:/Users/Luke/Downloads/edgedriver_win32/msedgedriver.exe")
# launch chromium/initialize driver settings (headless)
#options = EdgeOptions()
#options.use_chromium = True
#options.add_argument("headless")
#options.add_argument("disable-gpu")
# start the driver
browser=webdriver.Edge("C:/Users/Luke/Downloads/edgedriver_win32/msedgedriver.exe")
#browser=Edge("C:/Users/Luke/Downloads/edgedriver_win32/msedgedriver.exe",options=options)

data=[]
ticker_ii=7
# generate and retreive url
ticker=tickers[ticker_ii]
url='https://www.nasdaq.com/market-activity/stocks/{}/press-releases'.format(ticker.lower())
browser.get(url)
out_hrefs=[]
page=0
maxpages=1
# apply methods to get desired info from elements
# there's still a lot of bugs to work out
while True:
    try:
        # get url
        #browser.get(url)
        # go to specific start point 
        browser.execute_script("window.scrollTo(0,330)")
        # find the required href elements
        a_eles = browser.find_elements_by_xpath("//a[@class='quote-press-release__link']")
        browser.execute_script("arguments[0].scrollIntoView();", a_eles[0])
        # then go up a little
        browser.execute_script("window.scrollBy(0,-100)")
        # append them to list
        for ii in range(len(a_eles)):
            # try block 1
            # gets around stale exception errors
            unclicked0=True
            tries0=0
            while unclicked0 and tries0<10:
                try:
                    href = a_eles[ii].get_attribute("href")
                    out_hrefs.append(href)
                    unclicked0=False
                except:
                    tries0+=1
                    continue
        # get max page, check to see if we're past it
        # only do on first page
        if page==1:
            allpages=browser.find_elements_by_xpath("//button[@class='pagination__page']")
            #for npage in allpages:
                #if int(npage.text)>maxpages:
                    #maxpages=int(npage.text)
            maxpages=int(allpages[-1].text)
            # ALSO click the annoying button out of the way - doesnt work
            # make sure button is clickable
            #WebDriverWait(browser, 10).until(expected_conditions.visibility_of_element_located((By.CLASS_NAME,'onetrust-close-btn-handler onetrust-close-btn-ui banner-close-button ot-close-icon')))
            # click the next arrow
            #browser.find_element(By.CLASS_NAME,'onetrust-close-btn-handler onetrust-close-btn-ui banner-close-button ot-close-icon').click()
            #print('supposedly clicked x')
        # if we're over max pages then we're done
        if page==maxpages:
            break
        if page<maxpages:
            # try block 2
            unclicked1=True
            tries1=0
            while unclicked1 and tries1<10:
                try:
                    # get the button
                    nextbutton=browser.find_element(By.CLASS_NAME,'pagination__next')
                    browser.execute_script("arguments[0].scrollIntoView();", nextbutton)
                    # then go up a little
                    browser.execute_script("window.scrollBy(0,50)")
                    # make sure next arrow is clickable
                    WebDriverWait(browser, 10).until(expected_conditions.visibility_of_element_located((By.CLASS_NAME,'pagination__next')))
                    # click the next arrow
                    browser.find_element(By.CLASS_NAME,'pagination__next').click()
                    page+=1
                    unclicked1=False
                except:
                    tries1+=1
                    continue
        
    except IndexError:
        # no press releases on the site
        out_hrefs.append([ticker,np.nan])
        break
browser.close()

# export to external list for this ticker
file_path='C:/Users/Luke/OneDrive/Documents/PyScripts/hotshot finance type shit/ticker_releasedata/raw/%s.txt' % ticker

file=open(file_path, 'w')
for href in out_hrefs:
    file.write(href)
    file.write('\n')
file.close()

## Works, useful data is out_hrefs
From here we should save this to a database after running it for each ticker, then have a separate filter for duplicates and for updating for new releases. This function is implemented in pr_fullpull.

In [13]:
'/ticker_releasedata/raw/%s.txt' % ticker

'/ticker_releasedata/raw/ABT.txt'

In [34]:
out_hrefs

['https://www.nasdaq.com/press-release/abbott-hosts-conference-call-for-fourth-quarter-earnings-2022-01-04',
 'https://www.nasdaq.com/press-release/abbotts-proclaimtm-xr-spinal-cord-stimulation-system-now-offers-expanded-mri',
 'https://www.nasdaq.com/press-release/abbott-to-present-at-j.p.-morgan-healthcare-conference-2021-12-21',
 'https://www.nasdaq.com/press-release/abbott-names-robert-b.-ford-chairman-of-the-board-miles-d.-white-to-retire-as',
 'https://www.nasdaq.com/press-release/abbott-increases-quarterly-dividend-for-50th-consecutive-year-2021-12-10',
 'https://www.nasdaq.com/press-release/abbott-launches-similacr-360-total-carer-with-exclusive-blend-of-five-hmo-prebiotics',
 'https://www.nasdaq.com/press-release/abbott-achieves-highest-score-in-its-industry-for-the-ninth-consecutive-year-on-the',
 'https://www.nasdaq.com/press-release/abbotts-new-leadless-pacemaker-system-meets-primary-endpoints-in-pivotal-trial-2021',
 'https://www.nasdaq.com/press-release/abbott-declares-39

In [3]:
# Check things
data

[['ABBV', nan, nan, nan]]

In [21]:
# export
new=pd.DataFrame(data)

In [22]:
new.to_csv('newsreleases_DATE.csv',index=False)

In [83]:
a=4; b=2
print(d_eles[a].text, d_eles[b].text)
d_eles[a].text>d_eles[b].text

JAN 12, 2021 FEB 2, 2021


True

In [8]:
# Trying requests_html
from requests_html import HTMLSession
session=HTMLSession()

r = session.get('https://www.nasdaq.com/market-activity/stocks/GLUU/press-releases')
r.html.render()

TooManyRedirects: Exceeded 30 redirects.

In [ ]:
# Using Splash is reccomended for loading JS content
# UPDATE: Splash don't work on Windows